## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-19-06-08-07 +0000,bbc,White House says Iran would be 'wise' to make ...,https://www.bbc.com/news/articles/c204zpz0lyro...
1,2026-02-19-06-02-53 +0000,nypost,Disturbing viral bodycam footage captures doze...,https://nypost.com/2026/02/19/us-news/viral-di...
2,2026-02-19-05-50-43 +0000,nypost,Pima County Sheriff’s office at center of Nanc...,https://nypost.com/2026/02/19/us-news/pima-cou...
3,2026-02-19-05-48-21 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2026/02/19/world/...
4,2026-02-19-05-47-00 +0000,wsj,Airbus Warns of Hit to A320 Jet Production Fro...,https://www.wsj.com/business/earnings/airbus-w...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2475/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
80,trump,28
34,new,21
210,california,19
62,avalanche,15
7,up,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
272,2026-02-18-15-33-00 +0000,wsj,An immigration court dropped the Trump adminis...,https://www.wsj.com/politics/policy/judge-drop...,105
325,2026-02-18-11-59-00 +0000,wsj,Stephen Colbert’s stinging criticism of Presid...,https://www.wsj.com/politics/how-stephen-colbe...,82
344,2026-02-18-11-00-00 +0000,nypost,Lara Trump reveals Trump’s unexpected reaction...,https://nypost.com/2026/02/18/us-news/lara-tru...,81
204,2026-02-18-19-06-00 +0000,wsj,NIH Director Dr. Jay Bhattacharya will step in...,https://www.wsj.com/politics/policy/rfk-jr-cdc...,78
297,2026-02-18-14-18-49 +0000,latimes,Eight skiers dead and one missing in deadliest...,https://www.latimes.com/california/story/2026-...,72


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
272,105,2026-02-18-15-33-00 +0000,wsj,An immigration court dropped the Trump adminis...,https://www.wsj.com/politics/policy/judge-drop...
297,72,2026-02-18-14-18-49 +0000,latimes,Eight skiers dead and one missing in deadliest...,https://www.latimes.com/california/story/2026-...
50,63,2026-02-19-02-27-00 +0000,wsj,The top Republican challenger to Rep. Tony Gon...,https://www.wsj.com/politics/elections/texas-t...
325,54,2026-02-18-11-59-00 +0000,wsj,Stephen Colbert’s stinging criticism of Presid...,https://www.wsj.com/politics/how-stephen-colbe...
329,51,2026-02-18-11-30-00 +0000,wsj,Quantum Capital Raises $2.3 Billion So Far for...,https://www.wsj.com/articles/quantum-capital-r...
194,51,2026-02-18-19-43-00 +0000,wsj,Fed officials signaled little appetite for red...,https://www.wsj.com/economy/central-banking/fe...
315,48,2026-02-18-12-56-39 +0000,cbc,No sign of a breakthrough in Russia-Ukraine ta...,https://www.cbc.ca/news/world/switzerland-russ...
0,47,2026-02-19-06-08-07 +0000,bbc,White House says Iran would be 'wise' to make ...,https://www.bbc.com/news/articles/c204zpz0lyro...
361,40,2026-02-18-08-36-28 +0000,wsj,"Stock Market News, Feb. 18, 2026: Tech Stocks ...",https://www.wsj.com/livecoverage/stock-market-...
204,39,2026-02-18-19-06-00 +0000,wsj,NIH Director Dr. Jay Bhattacharya will step in...,https://www.wsj.com/politics/policy/rfk-jr-cdc...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
